<a href="https://colab.research.google.com/github/hodaka/MakeTrimmingMap/blob/work/%E8%87%AA%E5%8B%95%E3%83%88%E3%83%AA%E3%83%9F%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人物を切り抜く準備としてのトリミングマップを作製するためのスクリプト


---
[参考URL](https://v-hramchenko.medium.com/how-to-cut-out-a-person-in-an-image-with-open-source-projects-e5e7f8798d5c)

1.   各種ライブラリのインポート

In [ ]:
# GPUサポートを備えたTensorライブラリ(PyTorch自体はインストール済みcolabにインストール済み？)
import torch
# ファイル操作用
import os
# 画像操作ライブラリ
import cv2
# ファイルパス操作用ライブラリ
import glob
# 機械学習用拡張ライブラリ
import numpy as np
# 計算結果をグラフィカルに表示するライブラリ
import matplotlib.pyplot as plt

# セグメンテーション用ライブラリ
from torchvision.models.segmentation import deeplabv3_resnet101
# 画像の前処理や学習済みモデルなどを提供するエコシステム
from torchvision import transforms
# ファイル操作関連ライブラリ
from google.colab import files
# デバック用画像表示用ライブラリ
from IPython.display import Image, display

# 輪郭切り抜き用ライブラリ
!git clone https://github.com/MarcoForte/FBA-Matting.git
%cd FBA-Matting

import sys
sys.path.append("./FBA_Matting")

from demo import np_to_torch, pred, scale_input
from dataloader import read_image, read_trimap
from networks.models import build_model

class Args:
    encoder = 'resnet50_GN_WS'
    decoder = 'fba_decoder'
    weights = 'FBA.pth'

args=Args()
# 学習用のモデルを構築
try:
    model = build_model(args)
except:
    !gdown  https://drive.google.com/uc?id=1T_oiKDE_biWf2kqexMEN7ObWqtXAzbB1
    model = build_model(args)

2.   グーグルドライブをマウント



In [ ]:
from google.colab import drive
drive.mount('/drive')

3.   セグメンテーション用ライブラリを定義



In [21]:
# 対象を取得するセグメンテーション用ライブラリを定義
def make_deeplab(device):
    deeplab = deeplabv3_resnet101(pretrained=True).to(device)
    deeplab.eval()
    return deeplab

device = torch.device("cpu")
deeplab = make_deeplab(device)

4.   トリミング対象のファイルを表示して確認




In [ ]:
# アップロード機能を利用してファイルをアップロードする想定
f_name = "/content/206.500_1000.jpg"
img_orig = cv2.imread(f_name, 1)
plt.imshow(img_orig[:, :, ::-1])
plt.show()



5.   対象イメージをグレイスケールに


In [ ]:
k = min(1.0, 1024/max(img_orig.shape[0], img_orig.shape[1]))
img = cv2.resize(img_orig, None, fx=k, fy=k, interpolation=cv2.INTER_LANCZOS4)
plt.imshow(img[:, :, ::-1])
plt.show()

6.   人物を指定して対象部分を白それ以外を黒に


In [ ]:
deeplab_preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def apply_deeplab(deeplab, img, device):
    input_tensor = deeplab_preprocess(img)
    input_batch = input_tensor.unsqueeze(0)
    with torch.no_grad():
        output = deeplab(input_batch.to(device))['out'][0]
    output_predictions = output.argmax(0).cpu().numpy()
    return (output_predictions == 15)

mask = apply_deeplab(deeplab, img, device)

plt.imshow(mask, cmap="gray")
plt.show()

7.   トリミングマップ用に縁取りを実施



In [ ]:
trimap = np.zeros((mask.shape[0], mask.shape[1], 2))
trimap[:, :, 1] = mask > 0
trimap[:, :, 0] = mask == 0
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(25,25))

trimap[:, :, 0] = cv2.erode(trimap[:, :, 0], kernel)
trimap[:, :, 1] = cv2.erode(trimap[:, :, 1], kernel)

trimap_im =  trimap[:,:,1] + (1-np.sum(trimap,-1))/2
plt.imshow(trimap_im, cmap='gray', vmin=0, vmax=1)
plt.show()



8.   FBAモデルを使用して切り抜き



In [ ]:
fg, bg, alpha = pred((img/255.0)[:, :, ::-1], trimap, model)

img_ = img_orig.astype(np.float32)/255
alpha_ = cv2.resize(alpha, (img_.shape[1], img_.shape[0]), cv2.INTER_LANCZOS4)
fg_alpha = np.concatenate([img_, alpha_[:, :, np.newaxis]], axis=2)
cv2.imwrite(f_name + ".png", (fg_alpha*255).astype(np.uint8))
